In [8]:
import numpy as np
import damask
import yaml
import shutil
import os
import copy
from math import * 
os.getcwd()
print("The current directory is: ")
print(os.getcwd())

The current directory is: 
c:\Users\springnuance\Desktop\DAMASK3-Processing-Project


In [9]:
# https://damask.mpie.de/documentation/file_formats/grid_solver.html

# Do not modify these variables below
dir = "convert_DAMASK2_to_DAMASK3"
inputFile = "tensionX.load"
outputFile = "tensionX.yaml"
inputDir = f"{dir}/{inputFile}"
outputDir = f"{dir}/{outputFile}"

tension_convert = np.load("paramsMapping/tension_convert.npy", allow_pickle=True).tolist()

def is_float(string):
    try:
        float(string)
        return True
    except ValueError:
        return False

def convert_to_float(string):
    if string == "*":
        return "x"
    elif is_float(string):
        return float(string)
    else:
        return string

    
def check_keyword(token):
    if token == "*":
        return False
    try:
        float(token)
        return False
    except ValueError:
        return True

In [10]:
dot_F_DAMASK2 = ["fdot", "Fdot", "dotF"]
dot_P_DAMASK2 = ["pdot", "Pdot", "dotP"]
P_DAMASK2 = ["P", "stress"]
F_DAMASK2 = ["F"]
time_DAMASK2 = ["t", "time", "delta"]
incs_DAMASK2 = ["N", "incs"]
freq_DAMASK2 = ["freq", "frequency", "outputfreq"]

load_case = damask.Config(solver={'mechanical':'spectral_basic'},
                          loadstep=[])

loadstepTemplate = {'boundary_conditions': 
                        {
                        'mechanical':{}
                        },
                    'discretization':
                        {
                        't': None,
                        'N': None
                        },
                    'f_out': None
                    }

def addMatrix(index, tokens):
    matrix = []
    for _ in range(3):
        row = []
        for _ in range(3):
            row.append(convert_to_float(tokens[index]))
            index += 1
        matrix.append(row)
    return index, matrix

# Reading tensionX.load file
with open(inputDir) as file:
    # For each line of the file
    for line in file:
        #print(line.strip())
        # Split the line into a list of tokens
        tokens = line.split()
        loadstep = copy.deepcopy(loadstepTemplate)
        index = 0
        while index < len(tokens):
            DAMASK2_keyword = tokens[index]
            if DAMASK2_keyword in dot_F_DAMASK2:
                DAMASK3_keyword = tension_convert[DAMASK2_keyword]
                # Create empty numpy array
                index, dot_F_matrix = addMatrix(index+1, tokens)
                loadstep["boundary_conditions"]['mechanical'][DAMASK3_keyword] = dot_F_matrix
                continue
            elif DAMASK2_keyword in dot_P_DAMASK2:
                DAMASK3_keyword = tension_convert[DAMASK2_keyword]
                index, dot_P_matrix = addMatrix(index+1, tokens)
                loadstep["boundary_conditions"]['mechanical'][DAMASK3_keyword] = dot_P_matrix.tolist()
                continue
            elif DAMASK2_keyword in P_DAMASK2:
                DAMASK3_keyword = tension_convert[DAMASK2_keyword]
                index, P_matrix = addMatrix(index+1, tokens)
                loadstep["boundary_conditions"]['mechanical'][DAMASK3_keyword] = P_matrix
                continue
            elif DAMASK2_keyword in F_DAMASK2:
                DAMASK3_keyword = tension_convert[DAMASK2_keyword]
                index, F_matrix = addMatrix(index+1, tokens)
                loadstep["boundary_conditions"]['mechanical'][DAMASK3_keyword] = F_matrix
                continue
            elif DAMASK2_keyword in time_DAMASK2:
                DAMASK3_keyword = tension_convert[DAMASK2_keyword]
                index += 1
                loadstep["discretization"][DAMASK3_keyword] = convert_to_float(tokens[index])
                index += 1
                continue
            elif DAMASK2_keyword in incs_DAMASK2:
                DAMASK3_keyword = tension_convert[DAMASK2_keyword]
                index += 1
                loadstep["discretization"][DAMASK3_keyword] = int(tokens[index])
                index += 1
                continue
            elif DAMASK2_keyword in freq_DAMASK2:
                DAMASK3_keyword = tension_convert[DAMASK2_keyword]
                index += 1
                loadstep["f_out"] = int(tokens[index])
                index += 1
                continue

        load_case['loadstep'].append(loadstep)

load_case.save(outputDir, default_flow_style=None, sort_keys=False)
print("Saving the file tensionX.yaml in the directory convert_load_to_yaml")

Saving the file tensionX.yaml in the directory convert_load_to_yaml
